# Блок №5 Дополнительный проект

В папке subsid (`shared/homeworks/python_ds_miniprojects/5_subsid`) находятся файлы (`tm_sales_1`, `tm_sales_2`, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): `FILIAL_ID`, `SUBS_ID`, `PROD_ID`, `ACT_DTTM`.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

1. сотрудники телемаркетинга не всегда указывают полный id, если `'id'` нет в начале SUBS_ID, то нужно его добавить
2. поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
3. продажа не засчитывается, если отключение (`END_DTTM`) произошло меньше чем через 5 минут после подключения (`START_DTTM`)
4. если в файле с продажами встречается строка без указанного `SUBS_ID`, она пропускается

Сохраните результат в датафрэйм с разделителем `;`, содержащий корректные подключения.

Обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр `format`.

Пример содержания итогового файла (колонка difference это разница между `START_DTTM` и `END_DTTM`):
```
ACT_DTTM;FILIAL_ID;difference
20-03-2020 14:59;1;255 days 09:01:00
```

Для проверки результатов введите `SUBS_ID` из полученного датасета в порядке возрастания, через запятую с пробелом. Например:
```
id1, id2
```

In [1]:
# Импорт библиотек
import pandas as pd
import os
from pathlib import Path

In [2]:
# Путь до папки с файлами
path = '/home/jupyter-a.stamati-16/shared/homeworks/python_ds_miniprojects/5_subsid/subsid'

In [5]:
# Содержание папки
os.listdir(path)

['prod_activations_logs.csv',
 'tm_sales_1.csv',
 'tm_sales_2.csv',
 'tm_sales_3.csv']

In [6]:
# Отберем файлы с продажами продуктов
tms = Path(path).glob('tm*.csv')
# list(Path(path).glob('tm*'))

In [7]:
# Создади пустой датафрейм, куда запишем все данные
df = pd.DataFrame()

In [8]:
# В цикле запишем все данные в пустой датафрейм
for tm in tms:
    temp_df = pd.read_csv(tm, sep=';')
    
    df = pd.concat((df, temp_df))

In [9]:
# Посмотрим на получившийся результат
df.head()

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
1,NaN,4,7421,12-03-2020 11:25


In [10]:
df.dtypes

SUBS_ID      object
FILIAL_ID     int64
PROD_ID       int64
ACT_DTTM     object
dtype: object

In [11]:
#сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
# Создадим функцию для добавления 'id', там где его нет в начале
def id_adder(value):
    if value.startswith('id'):
        return value
    return 'id' + value

In [15]:
# Удалим пустые значения
df.dropna(subset=['SUBS_ID'], inplace=True)

In [16]:
# Добавим 'id' в начало, применив функцию 
df.SUBS_ID = df.SUBS_ID.apply(id_adder)

In [18]:
# Посмотрим на рзультат
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
2,id8362218,7,9879,05-03-2020 11:42
3,id2185490,2,3210,16-03-2020 16:28
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


In [20]:
# Считаем датасет с логами
logs = list(Path(path).glob('*logs*.csv'))[0]
logs = pd.read_csv(logs, sep=';')

In [21]:
logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
1,id4651830,1954,20-03-2020 14:59,01-12-2020 00:00
5,id7646509,6431,19-03-2020 13:00,19-03-2020 13:03
2,id7461794,3310,20-03-2020 17:25,01-12-2020 00:00
3,id5416547,1743,17-03-2020 10:17,25-03-2020 11:00
4,id8238421,1859,01-03-2020 11:42,01-03-2020 11:43
3,id8641743,2752,20-03-2020 15:44,21-04-2020 15:44
2,id2185490,3210,16-03-2020 16:28,01-12-2020 00:00
6,id7642700,3020,15-03-2020 14:21,15-03-2020 23:42
2,id8741631,5677,19-03-2020 12:28,01-12-2020 00:00


In [22]:
# Соеденим два датасета df и logs
full_data = df.merge(logs, on=['SUBS_ID', 'PROD_ID'])

In [54]:
# Получим такой результат
full_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,20-03-2020 14:59,20-03-2020 14:59,01-12-2020 00:00
1,id7646509,5,6431,19-03-2020 13:00,19-03-2020 13:00,19-03-2020 13:03
2,id5416547,3,1743,17-03-2020 10:17,17-03-2020 10:17,25-03-2020 11:00
3,id2185490,2,3210,16-03-2020 16:28,16-03-2020 16:28,01-12-2020 00:00
4,id7642700,6,3020,15-03-2020 14:21,15-03-2020 14:21,15-03-2020 23:42


In [55]:
full_data.dtypes

SUBS_ID       object
FILIAL_ID      int64
PROD_ID        int64
ACT_DTTM      object
START_DTTM    object
END_DTTM      object
dtype: object

In [23]:
# Переведем колонки со временем в нужный формат
full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] = full_data[['ACT_DTTM', 'START_DTTM', 'END_DTTM']] \
                                                        .apply(pd.to_datetime, format='%d-%m-%Y %H:%M')

In [24]:
full_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00


In [25]:
full_data.dtypes

SUBS_ID               object
FILIAL_ID              int64
PROD_ID                int64
ACT_DTTM      datetime64[ns]
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [26]:
# Создадим колонку с разницой времени между подключением и отключением
full_data['difference'] = full_data.END_DTTM - full_data.START_DTTM

In [27]:
full_data

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [28]:
# Отберем строки, где разница больше 5 минут
greater_than_5_min = full_data.difference > '5m'

In [29]:
# Вынесем такие строки в новый датафейм
df = full_data.loc[greater_than_5_min]

In [30]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,id4651830,1,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
2,id5416547,3,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [31]:
# Можем записать полученные данные в csv файл
# df[['ACT_DTTM', 'FILIAL_ID', 'difference']].to_csv('my_tm.csv', sep=';', index=False)

In [32]:
# Выведем на печать id пользователей
df.SUBS_ID.str.cat(sep=', ')

'id4651830, id5416547, id2185490, id7642700'